In [1]:
import numpy
import urllib.request
import scipy.optimize
import random
from math import exp
from math import log

def parseData(fname):
    for l in urllib.request.urlopen(fname):
        yield eval(l)

print ("Reading data...")
data=list(parseData("http://jmcauley.ucsd.edu/cse190/data/beer/beer_50000.json"))
print ("done")

Reading data...
done


In [2]:
def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
              loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
    print ("ll =", loglikelihood)
    return -loglikelihood

In [3]:
# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    sig=[]
    for i in range(len(X)):
        logit=inner(X[i],theta)
        sig.append(1-sigmoid(logit))
    X=numpy.matrix(X).T
    sig=numpy.matrix(sig).T
    y_opposite=[i*-1 for i in y]
    ones=[1]*len(y)
    y_flip=[sum(n) for n in zip(*[ones,y_opposite])]
    y_flip=numpy.matrix(y_flip).T
    theta=numpy.matrix(theta).T
    final=X*(sig-y_flip)-2*lam*theta
    dl=final.flatten().tolist()[0]
    pass
    
    return numpy.array([-x for x in dl])


In [4]:
def feature(datum):
    feat=[]
    feat.append(1)
    feat.append(datum['beer/ABV'])
    feat.append(datum['review/taste'])
    return feat
X=[feature(d) for d in data]
y=['American IPA' in d['beer/style'] for d in data]

In [5]:
X_train = X[:int(len(X)/2)]
X_test = X[int(len(X)/2):]
y_train=y[:int(len(y)/2)]
y_test=y[int(len(y)/2):]

In [6]:
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, y_train, 1.0))
print ("Final log likelihood =", -l)

ll = -17328.679514
ll = -16341.2013892
ll = -13952.6362653
ll = -6951.99741346
ll = -7358.42557314
ll = -6910.14444962
ll = -6906.4039954
ll = -6892.46364043
ll = -6803.17967995
ll = -6787.73594283
ll = -6783.12927193
ll = -6782.88180995
ll = -6782.29658661
ll = -6780.54303709
ll = -6776.3875002
ll = -6766.54731729
ll = -6747.07517192
ll = -6718.50759262
ll = -6888.22077568
ll = -6716.20422101
ll = -6695.66660152
ll = -6691.58168109
ll = -6690.85883334
ll = -6690.83165269
ll = -6690.83160231
ll = -6690.83159755
Final log likelihood = -6690.83159755


In [12]:
X_test=numpy.matrix(X_test)
theta=numpy.matrix(theta).T
theta


matrix([[-1.77315687],
        [-0.4530953 ],
        [ 0.67198756]])

In [13]:
predict_test=X_test*theta
predict_test_answer=[1 if predict_test[i]>0 else 0 for i in range (len(predict_test))]
match_test=[(x==y) for x,y in zip(y_train,predict_test_answer)]
accuracy_test=sum(match_test)/len(match_test)
print ("Accuracy for test data = ",accuracy_test)

Accuracy for test data =  0.9136


In [14]:
print(theta)

[[-1.77315687]
 [-0.4530953 ]
 [ 0.67198756]]


Accuracy for test data =  0.91344
